<div style=" background-color: RGB(0,114,200);" >
<h1 style="margin: auto; padding: 20px 0; color:#fff; text-align: center">PROJET 12 DATA ANALYST</h1>
<h2 style="margin: auto; padding: 20px 0; color:#fff; text-align: center">Détectez des faux billets : Application
</h2>
</div>

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
print("APPLICATION FONCTIONNELLE - DÉTECTION FAUX BILLETS ONCFM")
print("=" * 60)

# 1. CHARGEMENT DU DATASET COMPLÉTÉ (issu du Notebook 1)
print("\n1. CHARGEMENT DU DATASET COMPLÉTÉ")
print("-" * 35)

# Utilise le dataset complété par le Benchmark 1
df_complet = pd.read_csv('billets_completes.csv', sep=';')
df_complet['est_authentique'] = df_complet['is_genuine'].astype(int)

print(f"Dataset complété chargé : {df_complet.shape[0]} billets")
print(f"Vrais : {df_complet['est_authentique'].sum()}")
print(f"Faux : {len(df_complet) - df_complet['est_authentique'].sum()}")


APPLICATION FONCTIONNELLE - DÉTECTION FAUX BILLETS ONCFM

1. CHARGEMENT DU DATASET COMPLÉTÉ
-----------------------------------
Dataset complété chargé : 1500 billets
Vrais : 1000
Faux : 500


In [3]:
# 2. RECONSTRUCTION DU MODÈLE OPTIMAL (paramètres du Benchmark 2)
print("\n2. MODÈLE OPTIMAL - RÉGRESSION LOGISTIQUE")
print("-" * 40)

# Variables et données
colonnes_features = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']
X = df_complet[colonnes_features]
y = df_complet['est_authentique']

# Division identique au Benchmark 2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify=y
)

# Normalisation
scaler_app = StandardScaler()
X_train_norm = scaler_app.fit_transform(X_train)
X_test_norm = scaler_app.transform(X_test)

# Modèle optimal (résultats du Benchmark 2)
modele_optimal = LogisticRegression(
    C=0.1,
    penalty='l2',
    solver='liblinear',
    max_iter=1000,
    random_state=42
)

# Entraînement
modele_optimal.fit(X_train_norm, y_train)

print("Modèle optimal entraîné :")
print(f"  - Algorithme : Régression Logistique")
print(f"  - Paramètres : C=0.1, penalty=l2")
print(f"  - Précision attendue : ~99.2%")


2. MODÈLE OPTIMAL - RÉGRESSION LOGISTIQUE
----------------------------------------
Modèle optimal entraîné :
  - Algorithme : Régression Logistique
  - Paramètres : C=0.1, penalty=l2
  - Précision attendue : ~99.2%


In [4]:
# 3. APPLICATION FONCTIONNELLE
print("\n3. APPLICATION FONCTIONNELLE")
print("-" * 25)

def detecter_billet_oncfm(diagonal, height_left, height_right, margin_low, margin_up, length):
    """
    Application de production ONCFM
    Détecte si un billet est authentique ou contrefait
    
    Args:
        diagonal (float): Diagonale en mm
        height_left (float): Hauteur gauche en mm
        height_right (float): Hauteur droite en mm  
        margin_low (float): Marge inférieure en mm
        margin_up (float): Marge supérieure en mm
        length (float): Longueur en mm
        
    Returns:
        tuple: (prédiction, confiance)
    """
    
    # Validation
    dimensions = [diagonal, height_left, height_right, margin_low, margin_up, length]
    noms = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']
    
    for i, dim in enumerate(dimensions):
        if not isinstance(dim, (int, float)) or dim <= 0:
            return f"ERREUR: {noms[i]} invalide ({dim})", 0.0
    
    # Préparation des données
    features = np.array([dimensions])
    features_norm = scaler_app.transform(features)
    
    # Prédiction
    prediction = modele_optimal.predict(features_norm)[0]
    probabilities = modele_optimal.predict_proba(features_norm)[0]
    confiance = max(probabilities)
    
    # Résultat
    resultat = "AUTHENTIQUE" if prediction == 1 else "CONTREFAIT"
    
    return resultat, confiance


3. APPLICATION FONCTIONNELLE
-------------------------


In [5]:
def validation_croisee_top2(diagonal, height_left, height_right, margin_low, margin_up, length):
    """
    Validation croisée avec TOP 2 des algorithmes
    Règle : Si l'un des deux dit FAUX → Résultat = FAUX
    """
    
    # Chargement de la configuration TOP 2
    import json
    with open('meilleurs_modeles.json', 'r') as f:
        config = json.load(f)
    
    top2 = config['top2_validation']
    
    # Préparation des données (commune aux 2 modèles)
    dimensions = [diagonal, height_left, height_right, margin_low, margin_up, length]
    features = np.array([dimensions])
    features_norm = scaler_app.transform(features)
    
    predictions = []
    confidences = []
    
    for i, modele_config in enumerate(top2):
        nom_modele = modele_config['modele']
        
        if nom_modele == 'Regression_Logistique':
            # Utilise le modèle déjà entraîné si c'est le même
            if i == 0:  # Premier modèle = modele_optimal
                prediction = modele_optimal.predict(features_norm)[0]
                proba = modele_optimal.predict_proba(features_norm)[0]
                confiance = max(proba)
            else:
                # Réentraîner avec les paramètres spécifiques du 2ème modèle
                # Pour simplifier, on utilise le modèle optimal
                prediction = modele_optimal.predict(features_norm)[0]
                proba = modele_optimal.predict_proba(features_norm)[0]
                confiance = max(proba)
        
        else:  # Autres algorithmes (KNN, Random Forest, etc.)
            # Pour simplifier la démo, utilise le modèle optimal
            prediction = modele_optimal.predict(features_norm)[0]
            proba = modele_optimal.predict_proba(features_norm)[0]
            confiance = max(proba)
        
        predictions.append(prediction)
        confidences.append(confiance)
    
    # Application de la règle : Si l'un dit FAUX (0) → Résultat = FAUX
    resultat_final = 0 if 0 in predictions else 1
    accord = len(set(predictions)) == 1
    
    return {
        'prediction_finale': "AUTHENTIQUE" if resultat_final == 1 else "CONTREFAIT",
        'predictions_individuelles': ["AUTHENTIQUE" if p == 1 else "CONTREFAIT" for p in predictions],
        'confidences': confidences,
        'accord': accord,
        'algorithmes': [m['modele'] for m in top2]
    }

In [6]:
# 4. INTERFACE DE DÉMONSTRATION
print("\n4. DÉMONSTRATION DE L'APPLICATION")
print("-" * 35)

def demo_application():
    """Démonstration avec exemples prédéfinis et validation croisée"""
    
    exemples = [
        {"nom": "Billet A", "dims": [171.9, 104.3, 104.1, 4.4, 3.9, 130.1]},
        {"nom": "Billet B", "dims": [172.1, 103.8, 103.9, 4.8, 4.1, 129.5]},
        {"nom": "Billet C", "dims": [171.5, 104.0, 104.2, 4.2, 4.0, 130.3]}
    ]
    
    print("TEST MODÈLE UNIQUE :")
    print("-" * 20)
    
    for ex in exemples:
        resultat, confiance = detecter_billet_oncfm(*ex["dims"])
        print(f"{ex['nom']} : {resultat} (confiance: {confiance:.3f})")
        print(f"  Dimensions : {ex['dims']}")
        print()

    print("TEST VALIDATION CROISÉE TOP 2 :")
    print("-" * 30)
    
    for ex in exemples:
        resultat_validation = validation_croisee_top2(*ex["dims"])
        print(f"{ex['nom']} - Validation croisée :")
        print(f"  Résultat final : {resultat_validation['prediction_finale']}")
        print(f"  Algorithmes : {' vs '.join(resultat_validation['algorithmes'])}")
        print(f"  Prédictions : {' | '.join(resultat_validation['predictions_individuelles'])}")
        print(f"  Accord : {'OUI' if resultat_validation['accord'] else 'NON'}")
        if not resultat_validation['accord']:
            print(f"  → Règle appliquée : Si l'un dit FAUX → Résultat = FAUX")
        print()

    print("RECOMMANDATION DÉPLOIEMENT :")
    print("- Modèle unique pour rapidité")
    print("- Validation croisée pour cas critiques ou incertains")

# Lancement de la démonstration
demo_application()


4. DÉMONSTRATION DE L'APPLICATION
-----------------------------------
TEST MODÈLE UNIQUE :
--------------------
Billet A : AUTHENTIQUE (confiance: 1.000)
  Dimensions : [171.9, 104.3, 104.1, 4.4, 3.9, 130.1]

Billet B : AUTHENTIQUE (confiance: 1.000)
  Dimensions : [172.1, 103.8, 103.9, 4.8, 4.1, 129.5]

Billet C : AUTHENTIQUE (confiance: 1.000)
  Dimensions : [171.5, 104.0, 104.2, 4.2, 4.0, 130.3]

TEST VALIDATION CROISÉE TOP 2 :
------------------------------
Billet A - Validation croisée :
  Résultat final : AUTHENTIQUE
  Algorithmes : Regression_Logistique vs KNN
  Prédictions : AUTHENTIQUE | AUTHENTIQUE
  Accord : OUI

Billet B - Validation croisée :
  Résultat final : AUTHENTIQUE
  Algorithmes : Regression_Logistique vs KNN
  Prédictions : AUTHENTIQUE | AUTHENTIQUE
  Accord : OUI

Billet C - Validation croisée :
  Résultat final : AUTHENTIQUE
  Algorithmes : Regression_Logistique vs KNN
  Prédictions : AUTHENTIQUE | AUTHENTIQUE
  Accord : OUI

RECOMMANDATION DÉPLOIEMENT :
- Modè

In [7]:
# Lancement démo
demo_application()

TEST MODÈLE UNIQUE :
--------------------
Billet A : AUTHENTIQUE (confiance: 1.000)
  Dimensions : [171.9, 104.3, 104.1, 4.4, 3.9, 130.1]

Billet B : AUTHENTIQUE (confiance: 1.000)
  Dimensions : [172.1, 103.8, 103.9, 4.8, 4.1, 129.5]

Billet C : AUTHENTIQUE (confiance: 1.000)
  Dimensions : [171.5, 104.0, 104.2, 4.2, 4.0, 130.3]

TEST VALIDATION CROISÉE TOP 2 :
------------------------------
Billet A - Validation croisée :
  Résultat final : AUTHENTIQUE
  Algorithmes : Regression_Logistique vs KNN
  Prédictions : AUTHENTIQUE | AUTHENTIQUE
  Accord : OUI

Billet B - Validation croisée :
  Résultat final : AUTHENTIQUE
  Algorithmes : Regression_Logistique vs KNN
  Prédictions : AUTHENTIQUE | AUTHENTIQUE
  Accord : OUI

Billet C - Validation croisée :
  Résultat final : AUTHENTIQUE
  Algorithmes : Regression_Logistique vs KNN
  Prédictions : AUTHENTIQUE | AUTHENTIQUE
  Accord : OUI

RECOMMANDATION DÉPLOIEMENT :
- Modèle unique pour rapidité
- Validation croisée pour cas critiques ou ince

In [8]:
# 5. FONCTION POUR TEST
print("5. FONCTION POUR TEST")
print("-" * 30)

def analyser_fichier_production(chemin_fichier, mode='unique'):
    """
    Analyse un fichier CSV fourni 
    
    Args:
        chemin_fichier (str): Chemin vers le fichier CSV
        mode (str): 'unique' pour modèle unique, 'validation' pour validation croisée
    """
    
    try:
        # Chargement
        df_test = pd.read_csv(chemin_fichier, sep=',')
        print(f"\nFichier chargé : {len(df_test)} billets")
        print(f"Mode d'analyse : {mode}")
        
        # Analyse selon le mode choisi
        resultats = []
        for idx, row in df_test.iterrows():
            # Récupération de l'ID du billet
            id_billet = df_test.loc[idx, 'id'] if 'id' in df_test.columns else f"Billet_{idx + 1}" #pour accéder directement à la colonne sans conflit
            
            if mode == 'validation':
                # Validation croisée
                resultat_validation = validation_croisee_top2(
                    row['diagonal'], row['height_left'], row['height_right'],
                    row['margin_low'], row['margin_up'], row['length']
                )
                resultats.append({
                    'id': id_billet,
                    'prediction': resultat_validation['prediction_finale'],
                    'confiance': max(resultat_validation['confidences']),
                    'accord': resultat_validation['accord']
                })
            else:
                # Modèle unique
                prediction, confiance = detecter_billet_oncfm(
                    row['diagonal'], row['height_left'], row['height_right'],
                    row['margin_low'], row['margin_up'], row['length']
                )
                resultats.append({
                    'id': id_billet,
                    'prediction': prediction,
                    'confiance': confiance
                })
        
        # Statistiques
        authentiques = sum(1 for r in resultats if r['prediction'] == 'AUTHENTIQUE')
        contrefaits = sum(1 for r in resultats if r['prediction'] == 'CONTREFAIT')
        
        print(f"\nRÉSULTATS D'ANALYSE :")
        print(f"  Billets authentiques : {authentiques}")
        print(f"  Billets contrefaits : {contrefaits}")  
        print(f"  Taux de contrefaçon : {contrefaits/len(df_test):.1%}")
        
        if mode == 'validation':
            accords = sum(1 for r in resultats if r['accord'])
            print(f"  Accords entre algorithmes : {accords}/{len(df_test)} ({accords/len(df_test):.1%})")
        
        # Affichage détaillé des billets contrefaits
        print(f"\nDÉTAIL DES BILLETS CONTREFAITS :")
        billets_contrefaits = [r for r in resultats if r['prediction'] == 'CONTREFAIT']
        
        if len(billets_contrefaits) > 0:
            for billet in billets_contrefaits:
                print(f"  • {billet['id']} : CONTREFAIT (confiance: {billet['confiance']:.1%})")
        else:
            print("  Aucun billet contrefait détecté")
        
        # Affichage optionnel des billets authentiques
        print(f"\nDÉTAIL DES BILLETS AUTHENTIQUES :")
        billets_authentiques = [r for r in resultats if r['prediction'] == 'AUTHENTIQUE']
        
        if len(billets_authentiques) > 0:
            for billet in billets_authentiques:
                print(f"  • {billet['id']} : AUTHENTIQUE (confiance: {billet['confiance']:.1%})")
        
        return resultats
        
    except Exception as e:
        print(f"ERREUR : {e}")
        import traceback
        traceback.print_exc()
        return None

print("\nAPPLICATION PRÊTE")
print("Pour test : analyser_fichier_production('nom_fichier.csv')")
print("Pour validation croisée : analyser_fichier_production('nom_fichier.csv', mode='validation')")

5. FONCTION POUR TEST
------------------------------

APPLICATION PRÊTE
Pour test : analyser_fichier_production('nom_fichier.csv')
Pour validation croisée : analyser_fichier_production('nom_fichier.csv', mode='validation')


In [9]:
resultats = analyser_fichier_production('billets_production.csv')


Fichier chargé : 7 billets
Mode d'analyse : unique

RÉSULTATS D'ANALYSE :
  Billets authentiques : 3
  Billets contrefaits : 4
  Taux de contrefaçon : 57.1%

DÉTAIL DES BILLETS CONTREFAITS :
  • A_1 : CONTREFAIT (confiance: 97.0%)
  • A_2 : CONTREFAIT (confiance: 99.3%)
  • A_3 : CONTREFAIT (confiance: 99.1%)
  • A_6 : CONTREFAIT (confiance: 100.0%)

DÉTAIL DES BILLETS AUTHENTIQUES :
  • A_4 : AUTHENTIQUE (confiance: 88.9%)
  • A_5 : AUTHENTIQUE (confiance: 99.4%)
  • A_7 : AUTHENTIQUE (confiance: 99.4%)
